In [1]:
from eventsourcing.encryption import CryptoRepository,InMemCryptoStore
from eventsourcing.repositories import EventStoreRepository
from eventsourcing.event_stores import InMemEventStore
from example.user import User
from datetime import date
from example.guid import guid

### Initialize the in-memory encryption key store, eventstore, and repositories

In [2]:
crypto_store = InMemCryptoStore()
event_store = InMemEventStore()

CryptoRepository.crypto_store = crypto_store
user_repo = EventStoreRepository[User](event_store, User)

### Create a user and show that its attributes are clear

In [3]:
user = User(guid(), "Paul", "Boulanger", date(1997,2,18))

print(user.id, user.first_name, user.last_name, user.date_of_birth.isoformat())


d2a69be0-c97e-40d6-a752-08c98c02f97a Paul Boulanger 1997-02-18


### Save the user in the eventstore

The eventstore and key store are both empty

In [4]:
print(event_store.current)
print(crypto_store.store)

{}
{}


Saving the user in the eventstore

In [5]:
await user_repo.save(user,0)


The events related to the user are saved in the eventstore and an encryption key linked to the user was created and stored in the encryption key store

In [6]:
print(event_store.current)
print(crypto_store.store)

{'user-d2a69be0-c97e-40d6-a752-08c98c02f97a': [<example.event_store.EventDescriptor object at 0x0000022E4CDDF250>]}
{'d2a69be0-c97e-40d6-a752-08c98c02f97a': b'f4_FIVBXmGItxLYKUWR_obw58uzHhIjPI1WvxFQ-LxE='}


### Showing the encryption

The personnal data (in this case: `first_name`, `last_name`, `month_of_birth` and `day_of_birth`) are encrypted in the eventstore

In [7]:
event_store.current[User.to_stream_id(user.id)][0].event_data

'{"id": "d2a69be0-c97e-40d6-a752-08c98c02f97a", "first_name": "encrypted_gAAAAABmjz0AAr5Knbakw9XYOTRlmxUfjuPv1ffUQim8YPszHt1cKipJvKHpSgSboQrTAyfQQdZOKgptXYfBGgVzGuIN0aUL9w==", "last_name": "encrypted_gAAAAABmjz0Aq0wP-dlLCNxy73U3wsO0OfesD3ADY_nkFoEYB7d1afdCDQ6krU7cdKQKzcA8F13wEEiqO0T-LXCJUng46cmGmQ==", "year_of_birth": 1997, "month_of_birth": "encrypted_gAAAAABmjz0AUnVI40v55fPWE6hP3VtMrofjoyrCKSbP3I4FcfSS1fESiTEAhBXZoJVmyV1IyMnihnm8BPQxulAVPWhQmVlTsQ==", "day_of_birth": "encrypted_gAAAAABmjz0AyhlO5T9eUQ6gKhYKuD_txc3aIVYwh1XICbCI-_MY6nGeCcHGUvTHUQlICWu_uJw8wI_niqeBQeJlleD6nKvxVA=="}'

Since there is an encryption key, we can retrieve the personnal informations of the user in a decrypted state

In [8]:
events = await event_store.get_events_for_aggregate(User.to_stream_id(user.id))
events

[UserCreated(id='d2a69be0-c97e-40d6-a752-08c98c02f97a', first_name='Paul', last_name='Boulanger', year_of_birth=1997, month_of_birth=2, day_of_birth=18)]

We delete the user's encryption key

In [9]:
crypto_store.remove(user.id)

Now the personnal informations are still encrypted

In [10]:
events = await event_store.get_events_for_aggregate(User.to_stream_id(user.id))
events

[UserCreated(id='d2a69be0-c97e-40d6-a752-08c98c02f97a', first_name='encrypted_gAAAAABmjz0AAr5Knbakw9XYOTRlmxUfjuPv1ffUQim8YPszHt1cKipJvKHpSgSboQrTAyfQQdZOKgptXYfBGgVzGuIN0aUL9w==', last_name='encrypted_gAAAAABmjz0Aq0wP-dlLCNxy73U3wsO0OfesD3ADY_nkFoEYB7d1afdCDQ6krU7cdKQKzcA8F13wEEiqO0T-LXCJUng46cmGmQ==', year_of_birth=1997, month_of_birth='encrypted_gAAAAABmjz0AUnVI40v55fPWE6hP3VtMrofjoyrCKSbP3I4FcfSS1fESiTEAhBXZoJVmyV1IyMnihnm8BPQxulAVPWhQmVlTsQ==', day_of_birth='encrypted_gAAAAABmjz0AyhlO5T9eUQ6gKhYKuD_txc3aIVYwh1XICbCI-_MY6nGeCcHGUvTHUQlICWu_uJw8wI_niqeBQeJlleD6nKvxVA==')]

When we load our user from the eventstore, the personnal information are encrypted or replaced by fake information

In [11]:
new_user = User()
new_user.loads_from_history(events)
print(new_user.id, new_user.first_name, new_user.last_name, new_user.date_of_birth.isoformat())

d2a69be0-c97e-40d6-a752-08c98c02f97a encrypted_gAAAAABmjz0AAr5Knbakw9XYOTRlmxUfjuPv1ffUQim8YPszHt1cKipJvKHpSgSboQrTAyfQQdZOKgptXYfBGgVzGuIN0aUL9w== encrypted_gAAAAABmjz0Aq0wP-dlLCNxy73U3wsO0OfesD3ADY_nkFoEYB7d1afdCDQ6krU7cdKQKzcA8F13wEEiqO0T-LXCJUng46cmGmQ== 1997-01-01
